<a href="https://colab.research.google.com/github/MoonRiyadh/Neural-Symbolic-Computing/blob/master/Copy_of_NSC_model1_c.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torchtext==0.7

     |████████████████████████████████| 4.5MB 5.2MB/s 
     |████████████████████████████████| 1.1MB 54.0MB/s 
  Found existing installation: torchtext 0.3.1
    Uninstalling torchtext-0.3.1:
      Successfully uninstalled torchtext-0.3.1


In [ ]:
# imports
import torch 
import torchtext
import functools, operator
import os
from torchtext import data
from nltk import regexp_tokenize
from torch import nn, functional



BATCH_SIZE = 128
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(device)

## Data preparation

In [ ]:
# directory to data
dirname = "/content/drive/My Drive/Colab Notebooks/NSC_Project/Project.zip (Unzipped Files)"

In [ ]:
SAMPLE_LINES = 4

# The torchtext.data.Dataset class is not needed but is useful
class CustomDataset(torchtext.data.Dataset):
    def __init__(self, path, fields, **kwargs):
        """
        paths:
            path to data
        fields:
            tuple of Field objects (see torchtext.data.Field)
        """

        if not isinstance(fields[0], (tuple, list)):
            fields = [('input', fields[0]), ('output', fields[1])]

        with open(path) as f:
            lines = f.readlines()

        if len(lines) % 4 != 0:
            raise Exception(f":(, incomplete sample in {path}")
        
        examples = []
        for i in range(0, len(lines), SAMPLE_LINES):
            y = list(lines[i + SAMPLE_LINES - 1].rstrip('\n'))
            x = functools.reduce(
                operator.add,
                lines[i:i+SAMPLE_LINES - 1]
            )

            examples.append(data.Example.fromlist(
                        [x, y], fields))
        
        super(CustomDataset, self).__init__(examples, fields, **kwargs)

    @classmethod
    def create_datasets(cls, train, test, validation, fields, **kwargs):
        """
        train:
            path to train data
        test:
            path to test data
        validation:
            path to validation data
        fields:
            tuple of Field objects (see torchtext.data.Field)
        """
        train_set = cls(path=train, fields=fields, **kwargs)
        test_set = cls(path=test, fields=fields, **kwargs)
        validation_set = cls(path=validation, fields=fields, **kwargs)


        return train_set, test_set, validation_set 

In [ ]:
custom_tokenizer = lambda s : regexp_tokenize(s, r'(var|=|[a-z]+\d*|\d+|[+-])' )

INPUT = data.Field(tokenize = custom_tokenizer,
            init_token = '<sos>',
            eos_token = '<eos>',
            lower = True)
OUTPUT = data.Field(tokenize = custom_tokenizer,
            init_token = '<sos>',
            eos_token = '<eos>',
            lower = True)

# get datasets
train_data, test_data, validation_data = CustomDataset.create_datasets(
    *(os.path.join(dirname, f"{name}_Data.txt") for name in "Training Test Validation".split()),
    fields=(INPUT, OUTPUT)
)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
# create vocabs
INPUT.build_vocab(train_data)
OUTPUT.build_vocab(train_data)

## Iterators/Dataloader

In [ ]:
# get iters
train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, validation_data, test_data),
    batch_size = BATCH_SIZE,
    device = device,
    sort_key=lambda x: len(x.input + x.output))

## Model

1.      ENC_EMB_DIM = 32
2.      DEC_EMB_DIM = 32
3.      ENC_HID_DIM = 64
4.      DEC_HID_DIM = 64
5.      ATTN_DIM = 8
6.      ENC_DROPOUT = 0.5
7.      DEC_DROPOUT = 0.5

In [ ]:
import random
from typing import Tuple

import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch import Tensor


class Encoder(nn.Module):
    def __init__(self,
                 input_dim: int,
                 emb_dim: int,
                 enc_hid_dim: int,
                 dec_hid_dim: int,
                 dropout: float):
        super().__init__()

        self.input_dim = input_dim
        self.emb_dim = emb_dim
        self.enc_hid_dim = enc_hid_dim
        self.dec_hid_dim = dec_hid_dim
        self.dropout = dropout

        self.embedding = nn.Embedding(input_dim, emb_dim)

        self.rnn = nn.GRU(emb_dim, enc_hid_dim, bidirectional = True)

        self.fc = nn.Linear(enc_hid_dim * 2, dec_hid_dim)

        self.dropout = nn.Dropout(dropout)

    def forward(self,
                src: Tensor) -> Tuple[Tensor]:

        embedded = self.dropout(self.embedding(src))

        outputs, hidden = self.rnn(embedded)

        hidden = torch.tanh(self.fc(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1)))

        return outputs, hidden


class Attention(nn.Module):
    def __init__(self,
                 enc_hid_dim: int,
                 dec_hid_dim: int,
                 attn_dim: int):
        super().__init__()

        self.enc_hid_dim = enc_hid_dim
        self.dec_hid_dim = dec_hid_dim

        self.attn_in = (enc_hid_dim * 2) + dec_hid_dim

        self.attn = nn.Linear(self.attn_in, attn_dim)

    def forward(self,
                decoder_hidden: Tensor,
                encoder_outputs: Tensor) -> Tensor:

        src_len = encoder_outputs.shape[0]

        repeated_decoder_hidden = decoder_hidden.unsqueeze(1).repeat(1, src_len, 1)

        encoder_outputs = encoder_outputs.permute(1, 0, 2)

        energy = torch.tanh(self.attn(torch.cat((
            repeated_decoder_hidden,
            encoder_outputs),
            dim = 2)))

        attention = torch.sum(energy, dim=2)

        return F.softmax(attention, dim=1)


class Decoder(nn.Module):
    def __init__(self,
                 output_dim: int,
                 emb_dim: int,
                 enc_hid_dim: int,
                 dec_hid_dim: int,
                 dropout: int,
                 attention: nn.Module):
        super().__init__()

        self.emb_dim = emb_dim
        self.enc_hid_dim = enc_hid_dim
        self.dec_hid_dim = dec_hid_dim
        self.output_dim = output_dim
        self.dropout = dropout
        self.attention = attention

        self.embedding = nn.Embedding(output_dim, emb_dim)

        self.rnn = nn.GRU((enc_hid_dim * 2) + emb_dim, dec_hid_dim)

        self.out = nn.Linear(self.attention.attn_in + emb_dim, output_dim)

        self.dropout = nn.Dropout(dropout)


    def _weighted_encoder_rep(self,
                              decoder_hidden: Tensor,
                              encoder_outputs: Tensor) -> Tensor:

        a = self.attention(decoder_hidden, encoder_outputs)

        a = a.unsqueeze(1)

        encoder_outputs = encoder_outputs.permute(1, 0, 2)

        weighted_encoder_rep = torch.bmm(a, encoder_outputs)

        weighted_encoder_rep = weighted_encoder_rep.permute(1, 0, 2)

        return weighted_encoder_rep


    def forward(self,
                input: Tensor,
                decoder_hidden: Tensor,
                encoder_outputs: Tensor) -> Tuple[Tensor]:

        input = input.unsqueeze(0)

        embedded = self.dropout(self.embedding(input))

        weighted_encoder_rep = self._weighted_encoder_rep(decoder_hidden,
                                                          encoder_outputs)

        rnn_input = torch.cat((embedded, weighted_encoder_rep), dim = 2)

        output, decoder_hidden = self.rnn(rnn_input, decoder_hidden.unsqueeze(0))

        embedded = embedded.squeeze(0)
        output = output.squeeze(0)
        weighted_encoder_rep = weighted_encoder_rep.squeeze(0)

        output = self.out(torch.cat((output,
                                     weighted_encoder_rep,
                                     embedded), dim = 1))

        return output, decoder_hidden.squeeze(0)


class Seq2Seq(nn.Module):
    def __init__(self,
                 encoder: nn.Module,
                 decoder: nn.Module,
                 device: torch.device):
        super().__init__()

        self.encoder = encoder
        self.decoder = decoder
        self.device = device

    def forward(self,
                src: Tensor,
                trg: Tensor,
                teacher_forcing_ratio: float = 0.5) -> Tensor:

        batch_size = src.shape[1]
        max_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim

        outputs = torch.zeros(max_len, batch_size, trg_vocab_size).to(self.device)

        encoder_outputs, hidden = self.encoder(src)

        # first input to the decoder is the <sos> token
        output = trg[0,:]

        for t in range(1, max_len):
            output, hidden = self.decoder(output, hidden, encoder_outputs)
            outputs[t] = output
            teacher_force = random.random() < teacher_forcing_ratio
            top1 = output.max(1)[1]
            output = (trg[t] if teacher_force else top1)

        return outputs


INPUT_DIM = len(INPUT.vocab)
OUTPUT_DIM = len(OUTPUT.vocab)
# ENC_EMB_DIM = 256
# DEC_EMB_DIM = 256
# ENC_HID_DIM = 512
# DEC_HID_DIM = 512
# ATTN_DIM = 64
# ENC_DROPOUT = 0.5
# DEC_DROPOUT = 0.5

ENC_EMB_DIM = 32
DEC_EMB_DIM = 32
ENC_HID_DIM = 64
DEC_HID_DIM = 64
ATTN_DIM = 8
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

enc = Encoder(INPUT_DIM, ENC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, ENC_DROPOUT)

attn = Attention(ENC_HID_DIM, DEC_HID_DIM, ATTN_DIM)

dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, DEC_DROPOUT, attn)

model = Seq2Seq(enc, dec, device).to(device)


def init_weights(m: nn.Module):
    for name, param in m.named_parameters():
        if 'weight' in name:
            nn.init.normal_(param.data, mean=0, std=0.01)
        else:
            nn.init.constant_(param.data, 0)


model.apply(init_weights)

optimizer = optim.Adam(model.parameters())


def count_parameters(model: nn.Module):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


print(f'The model has {count_parameters(model):,} trainable parameters')

In [ ]:
PAD_IDX = OUTPUT.vocab.stoi['<pad>']

criterion = nn.CrossEntropyLoss(ignore_index=PAD_IDX)

In [ ]:
import time
import math


def train(model: nn.Module,
          iterator: data.BucketIterator,
          optimizer: optim.Optimizer,
          criterion: nn.Module,
          clip: float):

    model.train()

    epoch_loss = 0

    for _, batch in enumerate(iterator):

        src = batch.input
        trg = batch.output

        optimizer.zero_grad()

        output = model(src, trg)

        output = output[1:].view(-1, output.shape[-1])
        trg = trg[1:].view(-1)

        loss = criterion(output, trg)

        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)

        optimizer.step()

        epoch_loss += loss.item()

    return epoch_loss / len(iterator)


def evaluate(model: nn.Module,
             iterator: data.BucketIterator,
             criterion: nn.Module):

    model.eval()

    epoch_loss = 0

    with torch.no_grad():

        for _, batch in enumerate(iterator):

            src = batch.input
            trg = batch.output

            output = model(src, trg, 0) #turn off teacher forcing

            output = output[1:].view(-1, output.shape[-1])
            trg = trg[1:].view(-1)

            loss = criterion(output, trg)

            epoch_loss += loss.item()

    return epoch_loss / len(iterator)


def epoch_time(start_time: int,
               end_time: int):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs


N_EPOCHS = 10
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()

    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)

    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

test_loss = evaluate(model, test_iterator, criterion)

print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')

/usr/local/lib/python3.6/dist-packages/torchtext/data/batch.py:23: UserWarning: Batch class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('{} class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.'.format(self.__class__.__name__), UserWarning)


Epoch: 01 | Time: 1m 45s
	Train Loss: 1.452 | Train PPL:   4.272
	 Val. Loss: 1.280 |  Val. PPL:   3.596


In [ ]:
# save
torch.save(model.state_dict(), os.path.join(dirname, "model_params.pyt"))

## test

In [ ]:
def output_to_sentence(output):
    return ' '.join((OUTPUT.vocab.itos[torch.argmax(x).item()] for x in output))

def test(x, y, add):
    if add:
        res = x+y
        op = '+'
    else:
        res = x-y
        op = '-'
    
    input_str =  f"var x = {x}\nvar y = {y}\nx{op}y\n"
    output_str = f"{res}\n"

    example = data.Example.fromlist([input_str, output_str], [('input', INPUT), ('output', OUTPUT)])

    b = data.Batch((example,), test_data, device)

    #print(b.output)

    model.eval()
    output = model.forward(b.input, b.output, 0)

    output = output[1:].view(-1, output.shape[-1])
    y = b.output[1:].view(-1)

    #print(output)
    #print(y)


    print("Expected:", ' '.join((OUTPUT.vocab.itos[x] for x in y)))
    #print([x.item() for x in b.output])
    print("Got:", output_to_sentence(output))
    #print([torch.argmax(x).item() for x in output])

test(950, 1930, True)